In [1]:
DATE = "2021-05-20"
TASK = "phage-k-pangenome"
DIR = mkpath("$(homedir())/$(DATE)-$(TASK)")

"/Users/cameronprybol/2021-05-20-phage-k-pangenome"

In [2]:
pkgs = [
"Graphs",
"MetaGraphs",
"BioSequences",
"uCSV",
"DataFrames",
"FASTX",
"HTTP",
"CodecZlib",
"DataStructures",
"Revise",
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/.julia/dev/Mycelia/Project.toml`
No Changes to `~/.julia/dev/Mycelia/Manifest.toml`
┌ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
└ @ Base loading.jl:1278


In [3]:
ncbi_staph_phage_metadata = DataFrames.DataFrame(uCSV.read("$(dirname(pwd()))/metadata/ncbi-staph-phage.csv", header=1, quotes='"', typedetectrows=100)...)
ncbi_staph_phage_metadata = ncbi_staph_phage_metadata[ncbi_staph_phage_metadata[!, "Nuc_Completeness"] .== "complete", :]
ncbi_staph_phage_metadata = ncbi_staph_phage_metadata[ncbi_staph_phage_metadata[!, "Sequence_Type"] .== "RefSeq", :]
ncbi_staph_phage_metadata = ncbi_staph_phage_metadata[ncbi_staph_phage_metadata[!, "Genus"] .== "Kayvirus", :];
show(ncbi_staph_phage_metadata[1:3, :], allcols=true)

3×21 DataFrame
│ Row │ Accession │ SRA_Accession │
│     │ String    │ String        │
├─────┼───────────┼───────────────┤
│ 1   │ NC_047720 │               │
│ 2   │ NC_047721 │               │
│ 3   │ NC_047722 │               │

│ Row │ Submitters                                                                                                                                                                        │
│     │ String                                                                                                                                                                            │
├─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 1   │ Vandersteegen,K., Mattheus,W., Ceyssens,P.J., Bilocq,F., De Vos,D., Pirnay,J.P., Noben,J.P., Merabishvili,M., Lipinska,U., Hermans,K., Lavigne,R.                                 │
│ 2   │ Kirby,A.

In [7]:
# for k in [21, 31, 51]
k = 21
#     @show k
# for n_genomes in [2^i for i in 0:4]
n_genomes = 2
@show n_genomes
accessions = ncbi_staph_phage_metadata[1:n_genomes, "Accession"]
fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
KMER_TYPE = BioSequences.BigDNAMer{k}
@time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)
@time simplified_graph = Mycelia.simplify_kmer_graph(kmer_graph)
# Mycelia.graph_to_gfa(simplified_graph, "$(DIR)/simplified-kayvirus.$(k).$(n_genomes).gfa")
#     end
# end

n_genomes = 2


┌ Info: assessing kmers
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1773
┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1787


 13.076803 seconds (31.25 M allocations: 2.571 GiB, 32.89% gc time)


┌ Info: simplifying kmer graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2034
┌ Info: resolving untigs...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2035


  0.309621 seconds (2.59 M allocations: 89.213 MiB)


┌ Info: determining untig orientations...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2037
┌ Info: initializing graph node metadata
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2041
┌ Info: creating simplified untigs to resolve connections
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2051
┌ Info: resolving connections
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2059
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


  1.795832 seconds (7.61 M allocations: 415.919 MiB, 47.92% gc time)


{233, 618} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [22]:
# 11, 17 is too small
for k in [21, 31, 51]
    @show k
    for n_genomes in [2^i for i in 0:4]
        @show n_genomes
        outgraph = "$(DIR)/simplified-kayvirus.k_$(k).n-genomes_$(n_genomes).gfa"
        if !isfile(outgraph)
            accessions = ncbi_staph_phage_metadata[1:n_genomes, "Accession"]
            fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
            KMER_TYPE = BioSequences.BigDNAMer{k}
            @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)
            @time simplified_graph = Mycelia.simplify_kmer_graph(kmer_graph)
            Mycelia.graph_to_gfa(simplified_graph, outgraph)
        end
    end
end

k = 21
n_genomes = 1


┌ Info: assessing kmers
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1773
┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1787


  6.264016 seconds (19.96 M allocations: 1.848 GiB, 20.45% gc time)


┌ Info: simplifying kmer graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2034
┌ Info: resolving untigs...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2035


  0.339109 seconds (2.60 M allocations: 89.317 MiB)


...

153.680229 seconds (216.00 M allocations: 16.240 GiB, 46.00% gc time)


┌ Info: simplifying kmer graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2034
┌ Info: resolving untigs...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2035


  4.318895 seconds (8.29 M allocations: 293.837 MiB, 72.28% gc time)


┌ Info: determining untig orientations...
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2037
┌ Info: initializing graph node metadata
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2041
┌ Info: creating simplified untigs to resolve connections
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2051
┌ Info: resolving connections
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:2059
Progress: 100%|█████████████████████████████████████████| Time: 0:04:43


289.770169 seconds (3.20 G allocations: 83.027 GiB, 15.32% gc time)


^ Need to figure out a faster way to resolve these connections!